# Конвейер обработки текста

## Настройка рабочей среды

In [1]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.2                         
Location         C:\prj\nlp_spacy\venv\Lib\site-packages\spacy
Platform         Windows-10-10.0.22621-SP0     
Python version   3.11.4                        
Pipelines        en_core_web_sm (3.7.1), ru_core_news_sm (3.7.0)


In [0]:
#!python -m spacy download en

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.

In [ ]:
#!python -m spacy download en_core_web_sm

You can now load the package via spacy.load('en_core_web_sm')

In [ ]:
# !python -m spacy download ru_core_news_sm

You can now load the package via spacy.load('ru_core_news_sm')

## Загрузка корпуса

In [2]:
import spacy

In [3]:
nlp_en = spacy.load('en_core_web_sm')
nlp_ru = spacy.load('ru_core_news_sm')

## Токенизация

In [6]:
doc = nlp_en(u'I am flying to Frisco')
print([w.text for w in doc])

['I', 'am', 'flying', 'to', 'Frisco']


In [7]:
doc = nlp_ru(u'Я лечу в Казань')
print([w.text for w in doc])

['Я', 'лечу', 'в', 'Казань']


## Лемматизация

In [8]:
doc = nlp_en(u'this product integrates both libraries for downloading and applying patches')
for token in doc:
    print(token.text, token.lemma_)

this this
product product
integrates integrate
both both
libraries library
for for
downloading download
and and
applying apply
patches patch


In [9]:
#!pip install tabulate

In [10]:
from tabulate import tabulate

In [11]:
doc = nlp_ru(u'этот продукт объединяет обе библиотеки для загрузки и применения исправлений.')
print(tabulate([[token.text, token.lemma_] for token in doc], headers=['исх', 'лемма']))
# for token in doc:
#     print(token.text,'\t', token.lemma_)

исх          лемма
-----------  -----------
этот         этот
продукт      продукт
объединяет   объединять
обе          оба
библиотеки   библиотека
для          для
загрузки     загрузка
и            и
применения   применение
исправлений  исправление
.            .


## Использование лемматизации для распознавания смысла

In [18]:
from spacy.symbols import ORTH, LEMMA, NORM

In [25]:
doc = nlp_en(u'I am flying to Frisco')
print([w.text for w in doc])

nlp_en.get_pipe("attribute_ruler").add([[{"TEXT": "Frisco"}]], {"LEMMA": "San Francisco"})
# special_case = [{ORTH: u'Frisco', LEMMA: u'San Francisco'}]
# nlp_en.tokenizer.add_special_case(u'Frisco', special_case)
print([w.lemma_ for w in nlp_en(u'I am flying to Frisco')])

['I', 'am', 'flying', 'to', 'Frisco']
['I', 'be', 'fly', 'to', 'San Francisco']


In [27]:
doc = nlp_ru(u'Я лечу в Питер')
print([w.text for w in doc])

nlp_ru.get_pipe("attribute_ruler").add([[{"TEXT": "Питер"}]], {"LEMMA": "Санкт - Петербург"})
print([w.lemma_ for w in nlp_ru(u'Я лечу в Питер')])

['Я', 'лечу', 'в', 'Питер']
['я', 'лечу', 'в', 'Санкт - Петербург']


## Поиск соответствующих глаголов с помощью тегов частей речи

In [33]:
doc = nlp_en(u'I have flown to LA. Now I am flying to Frisco.')
print([w.text for w in doc if w.tag_== 'VBG' or w.tag_== 'VB'])

['flying']


In [34]:
print([w.tag_ for w in doc ])


['PRP', 'VBP', 'VBN', 'IN', 'NNP', '.', 'RB', 'PRP', 'VBP', 'VBG', 'IN', 'NNP', '.']


In [35]:
doc = nlp_ru(u'Я летал в Москву. Сейчас я лечу в Питер')
print([w.text for w in doc if w.tag_== 'VERB'])

['летал', 'лечу']


In [36]:
print([w.tag_ for w in doc ])


['PRON', 'VERB', 'ADP', 'PROPN', 'PUNCT', 'ADV', 'PRON', 'VERB', 'ADP', 'PROPN']


In [37]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Я PRON nsubj
летал VERB ROOT
в ADP case
Москву PROPN obl
. PUNCT punct
Сейчас ADV advmod
я PRON nsubj
лечу VERB ROOT
в ADP case
Питер PROPN obl


In [38]:
doc = nlp_ru(u'этот продукт объединяет обе библиотеки для загрузки и применения исправлений.')
print(tabulate([[token.text, token.pos_, token.dep_] for token in doc]))

-----------  -----  ----------
этот         DET    det
продукт      NOUN   nsubj
объединяет   VERB   ROOT
обе          NUM    nummod:gov
библиотеки   NOUN   obj
для          ADP    case
загрузки     NOUN   nmod
и            CCONJ  cc
применения   NOUN   conj
исправлений  NOUN   nmod
.            PUNCT  punct
-----------  -----  ----------


In [44]:
doc = nlp_ru(u'Я летал в Москву. Сейчас я лечу в Питер')
print(tabulate([[token.text, token.pos_, token.dep_] for token in doc]))

------  -----  ------
Я       PRON   nsubj
летал   VERB   ROOT
в       ADP    case
Москву  PROPN  obl
.       PUNCT  punct
Сейчас  ADV    advmod
я       PRON   nsubj
лечу    VERB   ROOT
в       ADP    case
Питер   PROPN  obl
------  -----  ------


In [45]:
print(tabulate([[token.head.text, token.dep_, token.text]  for token in doc]))

------  ------  ------
летал   nsubj   Я
летал   ROOT    летал
Москву  case    в
летал   obl     Москву
летал   punct   .
лечу    advmod  Сейчас
лечу    nsubj   я
лечу    ROOT    лечу
Питер   case    в
лечу    obl     Питер
------  ------  ------


В этом смысле многообещающе выглядят токены, маркированные
метками зависимостей ROOT и pobj (в данном примере они играют
ключевую роль в распознавании намерения). Как упоминалось ранее,
метка ROOT обозначает смысловой глагол предложения, а pobj в этом
примере отмечает сущность, которая в сочетании с глаголом резюми-
рует смысл всего высказывания.


In [47]:
for sent in doc.sents:
    print([w.text for w in sent if w.dep_ == 'ROOT' or w.dep_ == 'obl'])

['летал', 'Москву']
['лечу', 'Питер']


In [48]:
doc = nlp_en(u'I have flown to LA. Now I am flying to Frisco.')

In [50]:
for sent in doc.sents:
    print([w.text for w in sent if w.dep_ == 'ROOT' or w.dep_ == 'pobj'])

['flown', 'LA']
['flying', 'Frisco']


## Распознавание именованных сущностей

In [6]:
doc = nlp_en(u'I have flown to LA. Now I am flying to Frisco.')
for token in doc:
    if token.ent_type != 0:
        print(token.text, token.ent_type_)

LA GPE
Frisco ORG


In [8]:
doc = nlp_ru(u'Я живу в Сибири. Сейчас я лечу в Питер')
for token in doc:
    if token.ent_type != 0:
        print(token.text, token.ent_type_)

Сибири LOC
Питер LOC
